[View in Colaboratory](https://colab.research.google.com/github/zhangyi02/ZhangYi/blob/master/inceptionv3_scene15.ipynb)

In [2]:
! wget http://www-cvr.ai.uiuc.edu/ponce_grp/data/scene_categories/scene_categories.zip
! wget https://storage.googleapis.com/download.tensorflow.org/models/inception_dec_2015.zip
import zipfile
import os
def un_zip1():
    """unzip zip file"""
    zip_file = zipfile.ZipFile("inception_dec_2015.zip")
    if os.path.isdir("inception_dec_2015"):
        pass
    else:
        os.mkdir("inception_dec_2015")
    for names in zip_file.namelist():
        zip_file.extract(names,"inception_dec_2015")
    zip_file.close()
def un_zip2():
    """unzip zip file"""
    zip_file = zipfile.ZipFile("scene_categories.zip")
    if os.path.isdir("scene_categories"):
        pass
    else:
        os.mkdir("scene_categories")
    for names in zip_file.namelist():
        zip_file.extract(names,"scene_categories")
    zip_file.close()    
un_zip1()
un_zip2()

--2018-07-24 09:36:15--  http://www-cvr.ai.uiuc.edu/ponce_grp/data/scene_categories/scene_categories.zip
Resolving www-cvr.ai.uiuc.edu (www-cvr.ai.uiuc.edu)... 128.174.227.253
Connecting to www-cvr.ai.uiuc.edu (www-cvr.ai.uiuc.edu)|128.174.227.253|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86098604 (82M) [application/zip]
Saving to: ‘scene_categories.zip.2’

scene_categories.zi 100%[===================>]  82.11M  32.0MB/s    in 2.6s    

2018-07-24 09:36:18 (32.0 MB/s) - ‘scene_categories.zip.2’ saved [86098604/86098604]

--2018-07-24 09:36:19--  https://storage.googleapis.com/download.tensorflow.org/models/inception_dec_2015.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.28.128, 2607:f8b0:400e:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.28.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88631107 (85M) [application/zip]
Saving to: ‘inception_dec_2015.zip.2’

inception_d

In [3]:
import tensorflow as tf
tf.reset_default_graph()
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

Found GPU at: /device:GPU:0


In [7]:
#   Data
model_dir = './inception_dec_2015'
model_file = 'tensorflow_inception_graph.pb'
# os.mkdir("cache")
cache_dir = './cache'
input_data = './scene_categories'

datalab		    inception_dec_2015.zip  scene_categories.zip
inception_dec_2015  scene_categories
datalab		    inception_dec_2015.zip  scene_categories.zip
inception_dec_2015  scene_categories


In [13]:
import glob
import os.path
import random
import numpy as np
import tensorflow as tf
from tensorflow.python.platform import gfile
import pdb

#   Data
model_dir = './inception_dec_2015'
model_file = 'tensorflow_inception_graph.pb'
# os.mkdir("cache")
cache_dir = './cache'
input_data = './scene_categories'

# validation_percentage = 10
# test_percentage = 10

#   Model
bottleneck_tensor_size = 2048
bottleneck_tensor_name = 'pool_3/_reshape:0'
jpeg_data_tensor_name = 'DecodeJpeg/contents:0'

#   Parameters
learning_rate = 0.01
steps = 1000
batch = 100
checkpoint_every = 100
num_checkpoints = 5

# 从数据文件夹中读取所有的图片列表并按训练、验证、测试分开
def create_image_lists():
    result = {}  # 保存所有图像。key为类别名称。value也是字典，存储了所有的图片名称
    sub_dirs = [x[0] for x in os.walk(input_data)]  # 获取所有子目录
    is_root_dir = True  # 第一个目录为当前目录，需要忽略

    # 分别对每个子目录进行操作
    for sub_dir in sub_dirs:
        if is_root_dir:
            is_root_dir = False
            continue

        # 获取当前目录下的所有有效图片
        extensions = {'jpg', 'jpeg', 'JPG', 'JPEG'}
        file_list = []  # 存储所有图像
        dir_name = os.path.basename(sub_dir)  # 获取路径的最后一个目录名字
        for extension in extensions:
            file_glob = os.path.join(input_data, dir_name, '*.' + extension)
            file_list.extend(glob.glob(file_glob))
        if not file_list:
            continue

        # 将当前类别的图片随机分为训练数据集、测试数据集、验证数据集
        label_name = dir_name.lower()  # 通过目录名获取类别的名称
        # training_images = []
        # testing_images = []
        # validation_images = []
        training_images = random.sample(file_list, 100)
        for i in range(100):
            training_images[i] = os.path.basename(training_images[i])
        rest = list(set(file_list) - set(training_images))
        validation_images = random.sample(rest, 100)
        for i in range(100):
            validation_images[i] = os.path.basename(validation_images[i])
        testing_images = list(set(rest) - set(training_images))
        for i in range(len(testing_images)):
            testing_images[i] = os.path.basename(testing_images[i])
        # for file_name in file_list:
        #     base_name = os.path.basename(file_name)  # 获取该图片的名称
        #     pdb.set_trace()
        #     chance = np.random.randint(100)  # 随机产生100个数代表百分比
        #     if chance < validation_percentage:
        #         validation_images.append(base_name)
        #     elif chance < (validation_percentage + test_percentage):
        #         testing_images.append(base_name)
        #     else:
        #         training_images.append(base_name)

        # 将当前类别的数据集放入结果字典
        result[label_name] = {
            'dir': dir_name,
            'training': training_images,
            'testing': testing_images,
            'validation': validation_images
        }

    # 返回整理好的所有数据
    return result


# 通过类别名称、所属数据集、图片编号获取一张图片的地址
def get_image_path(image_lists, image_dir, label_name, index, category):
    label_lists = image_lists[label_name]  # 获取给定类别中的所有图片
    category_list = label_lists[category]  # 根据所属数据集的名称获取该集合中的全部图片
    mod_index = index % len(category_list)  # 规范图片的索引
    base_name = category_list[mod_index]  # 获取图片的文件名
    sub_dir = label_lists['dir']  # 获取当前类别的目录名
    full_path = os.path.join(image_dir, sub_dir, base_name)  # 图片的绝对路径
    return full_path


# 通过类别名称、所属数据集、图片编号获取特征向量值的地址
def get_bottleneck_path(image_lists, label_name, index, category):
    return get_image_path(image_lists, cache_dir, label_name, index,
                          category) + '.txt'


# 使用inception-v3处理图片获取特征向量
def run_bottleneck_on_image(sess, image_data, image_data_tensor,
                            bottleneck_tensor):
    bottleneck_values = sess.run(bottleneck_tensor,
                                 {image_data_tensor: image_data})
    bottleneck_values = np.squeeze(bottleneck_values)  # 将四维数组压缩成一维数组
    return bottleneck_values


# 获取一张图片经过inception-v3模型处理后的特征向量
def get_or_create_bottleneck(sess, image_lists, label_name, index, category,
                             jpeg_data_tensor, bottleneck_tensor):
    # 获取一张图片对应的特征向量文件的路径
    label_lists = image_lists[label_name]
    sub_dir = label_lists['dir']
    sub_dir_path = os.path.join(cache_dir, sub_dir)
    if not os.path.exists(sub_dir_path):
        os.makedirs(sub_dir_path)
    bottleneck_path = get_bottleneck_path(image_lists, label_name, index,
                                          category)

    # 如果该特征向量文件不存在，则通过inception-v3模型计算并保存
    if not os.path.exists(bottleneck_path):
        image_path = get_image_path(image_lists, input_data, label_name, index,
                                    category)  # 获取图片原始路径
        image_data = gfile.FastGFile(image_path, 'rb').read()  # 获取图片内容
        bottleneck_values = run_bottleneck_on_image(
            sess, image_data, jpeg_data_tensor,
            bottleneck_tensor)  # 通过inception-v3计算特征向量

        # 将特征向量存入文件
        bottleneck_string = ','.join(str(x) for x in bottleneck_values)
        with open(bottleneck_path, 'w') as bottleneck_file:
            bottleneck_file.write(bottleneck_string)
    else:
        # 否则直接从文件中获取图片的特征向量
        with open(bottleneck_path, 'r') as bottleneck_file:
            bottleneck_string = bottleneck_file.read()
        bottleneck_values = [float(x) for x in bottleneck_string.split(',')]

    # 返回得到的特征向量
    return bottleneck_values


# 随机获取一个batch图片作为训练数据
def get_random_cached_bottlenecks(sess, n_classes, image_lists, how_many,
                                  category, jpeg_data_tensor,
                                  bottleneck_tensor):
    bottlenecks = []
    ground_truths = []
    for _ in range(how_many):
        # 随机一个类别和图片编号加入当前的训练数据
        label_index = random.randrange(n_classes)
        label_name = list(image_lists.keys())[label_index]
        image_index = random.randrange(65535)
        bottleneck = get_or_create_bottleneck(
            sess, image_lists, label_name, image_index, category,
            jpeg_data_tensor, bottleneck_tensor)
        ground_truth = np.zeros(n_classes, dtype=np.float32)
        ground_truth[label_index] = 1.0
        bottlenecks.append(bottleneck)
        ground_truths.append(ground_truth)
    return bottlenecks, ground_truths


# 获取全部的测试数据
def get_test_bottlenecks(sess, image_lists, n_classes, jpeg_data_tensor,
                         bottleneck_tensor):
    bottlenecks = []
    ground_truths = []
    label_name_list = list(image_lists.keys())
    # 枚举所有的类别和每个类别中的测试图片
    for label_index, label_name in enumerate(label_name_list):
        category = 'testing'
        for index, unused_base_name in enumerate(
                image_lists[label_name][category]):
            bottleneck = get_or_create_bottleneck(
                sess, image_lists, label_name, index, category,
                jpeg_data_tensor, bottleneck_tensor)
            ground_truth = np.zeros(n_classes, dtype=np.float32)
            ground_truth[label_index] = 1.0
            bottlenecks.append(bottleneck)
            ground_truths.append(ground_truth)
    return bottlenecks, ground_truths


def main(_):
    # 读取所有的图片
    image_lists = create_image_lists()
    n_classes = len(image_lists.keys())

    with tf.Graph().as_default() as graph:
        # 读取训练好的inception-v3模型
        with gfile.FastGFile(os.path.join(model_dir, model_file), 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
            # 加载inception-v3模型，并返回数据输入张量和瓶颈层输出张量
            bottleneck_tensor, jpeg_data_tensor = tf.import_graph_def(
                graph_def,
                return_elements=[bottleneck_tensor_name, jpeg_data_tensor_name])

        # 定义新的神经网络输入
        bottleneck_input = tf.placeholder(
            tf.float32, [None, bottleneck_tensor_size],
            name='BottleneckInputPlaceholder')

        # 定义新的标准答案输入
        ground_truth_input = tf.placeholder(
            tf.float32, [None, n_classes], name='GroundTruthInput')

        # 定义一层全连接层解决新的图片分类问题
        with tf.name_scope('final_training_ops'):
            weights = tf.Variable(
                tf.truncated_normal(
                    [bottleneck_tensor_size, n_classes], stddev=0.1))
            biases = tf.Variable(tf.zeros([n_classes]))
            logits = tf.matmul(bottleneck_input, weights) + biases
            final_tensor = tf.nn.softmax(logits)

        # 定义交叉熵损失函数
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
            logits=logits, labels=ground_truth_input)
        cross_entropy_mean = tf.reduce_mean(cross_entropy)

        # 计算正确率
        with tf.name_scope('evaluation'):
            correct_prediction = tf.equal(
                tf.argmax(final_tensor, 1), tf.argmax(ground_truth_input, 1))
            evaluation_step = tf.reduce_mean(
                tf.cast(correct_prediction, tf.float32))

    # 训练过程
    with tf.Session(graph=graph) as sess:
        init = tf.global_variables_initializer().run()

        # 模型和摘要的保存目录
        import time
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(
            os.path.join(os.path.curdir, 'runs', timestamp))
        print('\nWriting to {}\n'.format(out_dir))
        # 损失值和正确率的摘要
        loss_summary = tf.summary.scalar('loss', cross_entropy_mean)
        acc_summary = tf.summary.scalar('accuracy', evaluation_step)
        # 训练摘要
        train_summary_op = tf.summary.merge([loss_summary, acc_summary])
        train_summary_dir = os.path.join(out_dir, 'summaries', 'train')
        train_summary_writer = tf.summary.FileWriter(train_summary_dir,
                                                     sess.graph)
        # 开发摘要
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, 'summaries', 'dev')
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)
        # 保存检查点
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, 'checkpoints'))
        checkpoint_prefix = os.path.join(checkpoint_dir, 'model')
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
            saver = tf.train.Saver(
                tf.global_variables(), max_to_keep=num_checkpoints)

        for i in range(steps):
            if i > 500:
                learning_rate = 0.001
            else:
                learning_rate = 0.01
            # 每次获取一个batch的训练数据
            train_bottlenecks, train_ground_truth = get_random_cached_bottlenecks(
                sess, n_classes, image_lists, batch, 'training',
                jpeg_data_tensor, bottleneck_tensor)
            train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy_mean)
            _, train_summaries = sess.run(
                [train_step, train_summary_op],
                feed_dict={
                    bottleneck_input: train_bottlenecks,
                    ground_truth_input: train_ground_truth
                })

            # 保存每步的摘要
            train_summary_writer.add_summary(train_summaries, i)

            # 在验证集上测试正确率
            if i % 100 == 0 or i + 1 == steps:
                validation_bottlenecks, validation_ground_truth = get_random_cached_bottlenecks(
                    sess, n_classes, image_lists, batch, 'validation',
                    jpeg_data_tensor, bottleneck_tensor)
                validation_accuracy, dev_summaries = sess.run(
                    [evaluation_step, dev_summary_op],
                    feed_dict={
                        bottleneck_input: validation_bottlenecks,
                        ground_truth_input: validation_ground_truth
                    })
                print(
                    'Step %d : Validation accuracy on random sampled %d examples = %.1f%%'
                    % (i, batch, validation_accuracy * 100))

            # 每隔checkpoint_every保存一次模型和测试摘要
            if i % checkpoint_every == 0:
                dev_summary_writer.add_summary(dev_summaries, i)
                path = saver.save(sess, checkpoint_prefix, global_step=i)
                print('Saved model checkpoint to {}\n'.format(path))

        # 最后在测试集上测试正确率
        test_bottlenecks, test_ground_truth = get_test_bottlenecks(
            sess, image_lists, n_classes, jpeg_data_tensor, bottleneck_tensor)
        test_accuracy = sess.run(
            evaluation_step,
            feed_dict={
                bottleneck_input: test_bottlenecks,
                ground_truth_input: test_ground_truth
            })
        print('Final test accuracy = %.1f%%' % (test_accuracy * 100))

        # 保存标签
        output_labels = os.path.join(out_dir, 'labels.txt')
        with tf.gfile.FastGFile(output_labels, 'w') as f:
            keys = list(image_lists.keys())
            for i in range(len(keys)):
                keys[i] = '%2d -> %s' % (i, keys[i])
            f.write('\n'.join(keys) + '\n')


if __name__ == '__main__':
    tf.app.run()



Writing to /content/runs/1532426620

Step 0 : Validation accuracy on random sampled 100 examples = 14.0%
Saved model checkpoint to /content/runs/1532426620/checkpoints/model-0

Step 100 : Validation accuracy on random sampled 100 examples = 58.0%
Saved model checkpoint to /content/runs/1532426620/checkpoints/model-100

Step 200 : Validation accuracy on random sampled 100 examples = 71.0%
Saved model checkpoint to /content/runs/1532426620/checkpoints/model-200

Step 300 : Validation accuracy on random sampled 100 examples = 75.0%
Saved model checkpoint to /content/runs/1532426620/checkpoints/model-300

Step 400 : Validation accuracy on random sampled 100 examples = 83.0%
Saved model checkpoint to /content/runs/1532426620/checkpoints/model-400

Step 500 : Validation accuracy on random sampled 100 examples = 86.0%
Saved model checkpoint to /content/runs/1532426620/checkpoints/model-500

Step 600 : Validation accuracy on random sampled 100 examples = 81.0%
Saved model checkpoint to /conte

SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
